In [5]:
import xml.etree.cElementTree as ET
import json
import codecs

In [2]:
add='D:/Anaconda3/mongodb-json-files/chicago.osm'


In [3]:
CREATED=['version', 'changeset', 'timestamp', 'user', 'uid']
ADDRESS=['addr:STE',
 'addr:city',
 'addr:country',
 'addr:housename',
 'addr:housenumber',
 'addr:interpolation',
 'addr:place',
 'addr:postcode',
 'addr:province',
 'addr:state',
 'addr:street',
 'addr:street:name',
 'addr:suite',
 'addr:unit']

In [4]:
# Via shape_element we convert every top level element tagged 'node' or 'way' into a python dictionary

def shape_element(element):
    dict_ = {}
    dict_['created']={}
    dict_['address']={}
    if element.tag == "node" or element.tag == "way" :
        if element.tag=='node':
            dict_['pos']=[0,0]
            dict_['type']='node'
        else:
            dict_['node_refs']=list()
            dict_['type']='way'
        for key1 in element.attrib.keys():
            if key1 in CREATED:
                dict_['created'][key1]=element.attrib[key1]
            elif key1=='lat':
                dict_['pos'][0]=float(element.attrib[key1])
            elif key1=='lon':
                dict_['pos'][1]=float(element.attrib[key1])
            else:
                dict_[key1]=element.attrib[key1]
        for sub_element in element.getchildren():
            if sub_element.tag=='nd':
                dict_['node_refs'].append(sub_element.attrib['ref'])
            if sub_element.tag=='tag':
                if sub_element.attrib['k'] in ADDRESS:
                    dict_['address'][sub_element.attrib['k'].split(':')[1]]=sub_element.attrib['v']
                else:
                    dict_[sub_element.attrib['k']]=sub_element.attrib['v']
        if dict_['address']=={}:
            dict_.pop('address')
                        

        
        
        return dict_
    else:
        return None


In [7]:
pretty=False
with codecs.open("chicago.json", "w") as fo:
    for _, element in ET.iterparse(add):
        el = shape_element(element)
        if el:
            if pretty:
                fo.write(json.dumps(el, indent=2)+"\n")
            else:
                fo.write(json.dumps(el) + "\n")
                
#chicago.json is approximately 2 gb

In [8]:
pretty=True
with codecs.open("chicago_with_indentation.json", "w") as fo:
    for _, element in ET.iterparse(add):
        el = shape_element(element)
        if el:
            if pretty:
                fo.write(json.dumps(el, indent=2)+"\n")
            else:
                fo.write(json.dumps(el) + "\n")
                
#chicago.json is approximately 2.55 gb
#The more indentation the larger resultinf file, obviously

In [9]:
#converting into mongodb

from pymongo import MongoClient
client = MongoClient()
client = MongoClient('mongodb://localhost:27017/')

In [10]:
#create a mongo database named 'chicago-osm' in local host

db = client["chicago-osm"]


In [11]:
#create collections named nodes and ways in chicago-osm

col_nodes = db["nodes"]
col_ways = db["ways"]

In [14]:
#inserting the documents into relevant collections

for event, element in ET.iterparse(add,events=("start",)):
    el = shape_element(element)
    if el:
        if element.tag=='node':
            col_nodes.insert_one(el)
        else:
            col_ways.insert_one(el)

In [15]:
counter=0
for event, element in ET.iterparse(add,events=("start",)):
    if element.tag=='node':
        counter=counter+1

In [16]:
counter

7679935

In [18]:
#total number of node documents in the collection
col_nodes.count_documents({})

7679935

In [19]:
#total number of way documents in the collection
col_ways.count_documents({})

1087021